In [ ]:
%tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/drive/')
import re 
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
Mounted at /content/drive/
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
import tarfile
import os
import json
import requests
import sys
import shutil
import re
from tqdm import tqdm, trange
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.python.client import device_lib
import time
from datetime import datetime
import csv
import argparse

# if in Google Colaboratory
try:
    from google.colab import drive
except:
    pass

from gpt_2_simple.src import model, sample, encoder, memory_saving_gradients
from gpt_2_simple.src.load_dataset import load_dataset, Sampler
from gpt_2_simple.src.accumulate import AccumulatingOptimizer

In [ ]:
names = {
  " ADI "  :  " ADIRA TAL ",  
  " BRN "  :  " MICHAEL BURNHAM ",
  " BRY "  :  " BRYCE ",
  " BK "   :  " BOOKER ",
  " COMP " :  " COMPUTER ",
  " COS "  :  " COSMO ",
  " DET "  :  " KEYLA DETMER ",
  " GAB "  :  " GABRIELLE BURNHAM ", 
  " GRG "  :  " PHILLIPA GEORGIOU ",
  " GRY "  :  " GREY TAL ",
  " ITH "  :  " ITHOR ",
  " TLR "  :  " TOLOR ", 
  " NIL "  :  " NILSSON ", 
  " OWO "  :  " JOANN OWOSEKUN ",
  " OSS "  :  " OSSYRA ", 
  " POL "  :  " DR.POLLARD ", 
  " RHYS " :  " GEN RHYS ",
  " SAR "  :  " SARU ", 
  " SEN "  :  " SENNA TAL ", 
  " STA "  :  " PAUL STAMETS ", 
  " TLY "  :  " TILLY ", 
  " NDY "  :  " CAPTAIN NDYOE ",
  " XI "   :  ' GAURDIAN TRILL ',
  " PAV "  :  " LEADER TRILL ",
  " VN "   :  " VANCE ",
  " KAR "  :  " CAPTAIN KARDASHEV ", 
  " RA "   :  " CAPTAIN RAHMA ", 
  " BAN "  :  " CAPTAIN BANDRA ", 
  " VK "   :  " V'KIR ",
  " NR "   :  " N'RAJ ",
  " SHR "  :  " SHIRA ",   
  " TR "   :  " T'RINA ",
  " ARL "  :  " AURELLIO ",
  " CR "   :  " CARL ", 
  " ISS HOLO " :  " SUKALS MOM ", 
  " KLY "  :  " KILLY ", 
  " SKL "  :  " SUKAL ", 
  " WL "   :  " LT WILLA ", 
  " OS "   :  " OS'IR  BARTENDER ",
  " ZAR "  :  " ZAREH ", 
  " HLM COMP " : " HELMENT COMPUTER ", 
  " ADM "  :  " ADMIRAL CORNWELL ",
  " ASH "  :  " ASH TYLER ", 
  " PKE "  :  " CHRISTOPHER PIKE ", 
  " SPK "  :  " SPOCK ", 
  " SRK "  :  " SAREK ", 
  " AMD "  :  " AMANDA ", 
  " LRL "  :  " L'RELL ", 
  " LEL "  :  " LELAND ",
  " NUM "  :  " NUMBER ONE ",
  " SIR " :   " SIRANNA ",
  " BRN DAD " : " BURNHAMS DAD ", 
  " CTRL " : " CONTROL ",
  " VIK "  :  " TENAVIK ",
  " KAM "  :  " KAMARAN GANT ",
  " CONN " :  " EVAN CONNOLLY ",
  " GAR "  :  " GARRISON ", 
  " TAL "  : " TALOS MAN ", 
  " TLSN " : " TALOSIAN ", 
  " PSY "  : " PYSCH ", 
  " SEC ONE " : "FIRST GAURD ", 
  " SEC TWO " : "SECOND GAURD ",
  " THR TRANS " :  " SECTION THIRTY-ONE TRANSPORTER ", 
  " ARIM " :  " AIRIAM ",
  " HUS " : " AIRIAM HUSBAND ",
  " PAT "  :  " PATAR ", 
  " ANDO"  : " ANDORAIN ADMIRAL ",
  " TEL "  :  " TELLARITE ADMIRAL ", 
  " KOL "  :  " KOL-SHA ",
  " UJI "  :  " UJILI MO'KAI ", 
  " CUL HOLO " : " HOLOGRAPHIC HUGH CULBER ", 
  " JL "  :  " PICARD ", 
  " FI "  :  " RAFFI ",
  " AGN " :  " ANGES ",
  " HG "  :  " HUGH ", 
  " SVN " :  " SEVEN ", 
  " ICH " :  " ICHEB ",
  " NRK " :  " NAREK ", 
  " SJ "  :  " SOJI ",
  " NRS " :  " NARISSA ", 
  " RK "  :  " WILL RIKER ", 
  " TR "  :  " DEANNA TROI ", 
  " EM "  :  " EMIL ", 
  " EN "  :  " EMERGENCY NAVIGATIONAL ",
  " EH "  :  " EMERGENCY HOSPITALITY ",
  " EMM " :  " EMMOT ", "'re"  : ' are', 
  " AR " :   "ARCANA ", "'ve" : " have",
  " ALT " :  "ALTON SOONG ", "'nt" : " not ",
  " ST "  :  " SUTRA ",  "'ll":' will',
  " CL "  :  " ADMIRAL CLANCY ",
  "'S" : '', ':' : '', ';' : '',
  "'" : '', ' BILL ': 'BILLUPS',
  ' MAR ' : ' BECKETT MARINER ', 
  ' RUTH ' : ' RUTHERFORD ',
  ' V CAP ' : ' VANCOUVER CAPTAIN ',
  ' CAP ' : 'CAPTAIN CAROL FREEMAN',
  ' RAN ' : ' JACK RANSOM',
  ' TEN ' : ' DVANA TENDI ',
  ' SHAX ' : 'SHAXS', 
  ' PHAMP ' : ' PEANUT HAMPER ',
  ' BARB ' : ' BARBARA BRINSON ',
  ' DOC ' : ' RON DOCENT ', 
  ' MIXTUS CIT: ' : ' MIXTUS CITIZEN', 
  ' AMI ' : ' CAPTAIN AMINA ', 
  ' DUR ' : ' DURGAN ', 
  ' DIV ' : ' SPECIALIST DIVISION 14',
  ' PRA ' : ' PRACHETT ',
  ' WAR ' : ' WARREN ',
  ' HALF ' : ' HALF-OLD HALF-YOUNG ',
  ' INT. ' : ' INTERIOR ', 
  ' LT. ' : ' LIEUTENANT ',
  ' LT ' : ' LIEUTENANT ',
  ' CMR. ' : ' COMMANDER ',
  ' CMR ' : ' COMMANDER ',
  ' POL ' : ' DOCTOR POLLARD ',
  ' DR.' : ' DOCTOR ',
  ' starlog ' : ' log ',
  ' EXT. ' : ' EXTERIOR '
  }

In [ ]:
to_drop = ['CONTINUED', 'CUT TO', 'FADE IN', 
           'FADE OUT', 'END OF TEASER', 'OMITTED', 'INTERCUT:', 
           'THE WRITING CREDITS MAY NOT BE FINAL',  'THE END',
           'FINAL DRAFT', 'FIRST DRAFT', 'THEME MUSIC',
           'END OF ACT', '/66', '/67', '/68', '/69', '/70', 
           '/71', '/72', '/73', '/74', '/75', '/76', '/77', 
           '/78', '/79', '/80', '/81', '/82', '/83', '/84', 
           '/85', '/86', '/87', '/88', '/89', '/90', '/91', 
           '/92', '/93', '/94', '/95', '/96', '/97', '/98', 
           '/99', '/00', '/01', '/02', '/03', '/04', '/05', 
           '/06', '/07', '/08', '/09', '/10', '/11', '/12',
           '/13', '/14', '/15', '/16', '/17', '/18', '/19', 
           '/20', '/21', 'END OF ACT', 'STAR TREK:', 'SPACE NINE:', 
           'VOYAGER:', 'PRISE:', 'GENERATION:', 'ACT ONE', 'ACT TWO', 
           'ACT THREE', 'ACT FOUR', 'ACT FIVE', 'ACT SIX', 'ACT SEVEN', 
           'ACT EIGHT', 'SCENE CHANGE', 'BEAT.', 'AS BEFORE.', 
           'NOTE:', 'DISSOLVE TO:', 'PART ONE', 'PART TWO', 'PART THREE', 
           'PART FOUR', 'THRU OMITTED', '(BEAT)', '(CONTINUING)',
           'FADE IN', 'FADE OUT', 'CUT TO', 'ANGLE ON',
           'NEW ANGLE', 'ANGLE EMPHASIZING', '(A BEAT)',
           ' POV ', 'PART ONE', 'PART TWO', 'PART THREE', 'PART FOUR',
           'ACT ONE', 'ACT TWO', 'ACT THREE', 'ACT FOUR', 'TEASER']


In [ ]:
def finetune(sess,
             dataset,
             steps=-1,
             model_name='124M',
             model_dir='models',
             combine=50000,
             batch_size=1,
             learning_rate=0.0001,
             accumulate_gradients=5,
             restore_from='latest',
             run_name='run1',
             checkpoint_dir='checkpoint',
             sample_every=100,
             sample_length=1023,
             sample_num=1,
             multi_gpu=False,
             save_every=1000,
             print_every=1,
             max_checkpoints=1,
             use_memory_saving_gradients=False,
             only_train_transformer_layers=False,
             optimizer='adam',
             overwrite=False):
    """Finetunes the model on the given dataset.

    Adapted from https://github.com/nshepperd/gpt-2/blob/finetuning/train.py.
    See that file for parameter definitions.
    """
    SAMPLE_DIR = 'samples'

    checkpoint_path = os.path.join(checkpoint_dir, run_name)

    def maketree(path):
        try:
            os.makedirs(path)
        except:
            pass

    maketree(checkpoint_path)
    files = [f for f in os.listdir(checkpoint_path)]
    for file in ['hparams.json', 'encoder.json', 'vocab.bpe']:
        try:
            shutil.copyfile(os.path.join(model_dir, model_name, file),
                            os.path.join(checkpoint_path, file))
        except FileNotFoundError as fnf_error:
            print("You need to download the GPT-2 model first via download_gpt2()")
            raise(fnf_error)

    enc = encoder.get_encoder(checkpoint_path)
    hparams = model.default_hparams()
    with open(os.path.join(checkpoint_path, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if sample_length > hparams.n_ctx:
        raise ValueError(
            "Can't get samples longer than window size: %s" % hparams.n_ctx)

    if model_name not in ['117M', '124M']:
        use_memory_saving_gradients = True
        only_train_transformer_layers = True
        accumulate_gradients = 1

    context = tf.compat.v1.placeholder(tf.int32, [batch_size, None])
    gpus = []

    if multi_gpu:
        gpus = get_available_gpus()

    output = model.model(hparams=hparams, X=context, gpus=gpus)
    loss = tf.reduce_mean(
        input_tensor=tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=context[:, 1:], logits=output['logits'][:, :-1]))

    tf_sample = sample.sample_sequence(
        hparams=hparams,
        length=sample_length,
        context=context,
        batch_size=batch_size,
        temperature=1.0,
        top_k=40)

    all_vars = [v for v in tf.compat.v1.trainable_variables() if 'model' in v.name]
    train_vars = [v for v in all_vars if '/h' in v.name] if only_train_transformer_layers else all_vars

    if optimizer == 'adam':
        opt = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate)

    if accumulate_gradients > 1:
        if use_memory_saving_gradients:
            exit("Memory saving gradients are not implemented for gradient accumulation yet.")
        opt = AccumulatingOptimizer(
            opt=opt,
            var_list=train_vars)
        opt_reset = opt.reset()
        opt_compute = opt.compute_gradients(loss)
        opt_apply = opt.apply_gradients()
        summary_loss = tf.compat.v1.summary.scalar('loss', opt_apply)
    else:
        if use_memory_saving_gradients:
            opt_grads = memory_saving_gradients.gradients(loss, train_vars)
        else:
            opt_grads = tf.gradients(ys=loss, xs=train_vars)
        opt_grads = list(zip(opt_grads, train_vars))
        opt_apply = opt.apply_gradients(opt_grads)
        summary_loss = tf.compat.v1.summary.scalar('loss', loss)

    summary_log = tf.compat.v1.summary.FileWriter(checkpoint_path)

    saver = tf.compat.v1.train.Saver(
        var_list=all_vars,
        max_to_keep=max_checkpoints)
    sess.run(tf.compat.v1.global_variables_initializer())

    if restore_from == 'latest':
        ckpt = tf.train.latest_checkpoint(checkpoint_path)
        if ckpt is None:
            # Get fresh GPT weights if new run.
            ckpt = tf.train.latest_checkpoint(
                os.path.join(model_dir, model_name))
    elif restore_from == 'fresh':
        ckpt = tf.train.latest_checkpoint(
            os.path.join(model_dir, model_name))
    else:
        ckpt = tf.train.latest_checkpoint(restore_from)
    print('Loading checkpoint', ckpt)
    saver.restore(sess, ckpt)

    print('Loading dataset...')
    chunks = load_dataset(enc, dataset, combine)
    data_sampler = Sampler(chunks)
    print('dataset has', data_sampler.total_size, 'tokens')
    print('Training...')

    counter = 1
    counter_path = os.path.join(checkpoint_path, 'counter')
    if os.path.exists(counter_path) and restore_from == 'latest':
        # Load the step number if we're resuming a run
        # Add 1 so we don't immediately try to save again
        with open(counter_path, 'r') as fp:
            counter = int(fp.read()) + 1
    counter_base = counter

    def save():
        maketree(checkpoint_path)
        print(
            'Saving',
            os.path.join(checkpoint_path,
                         'model-{}').format(counter-1))
        saver.save(
            sess,
            os.path.join(checkpoint_path, 'model'),
            global_step=counter-1)
        with open(counter_path, 'w') as fp:
            fp.write(str(counter-1) + '\n')

    def generate_samples():
        context_tokens = data_sampler.sample(1)
        all_text = []
        index = 0
        while index < sample_num:
            out = sess.run(
                tf_sample,
                feed_dict={context: batch_size * [context_tokens]})
            for i in range(min(sample_num - index, batch_size)):
                text = enc.decode(out[i])
                text = '======== SAMPLE {} ========\n{}\n'.format(
                    index + 1, text)
                all_text.append(text)
                index += 1
        print(text)
        maketree(os.path.join(SAMPLE_DIR, run_name))
        with open(
                os.path.join(SAMPLE_DIR, run_name,
                             'samples-{}').format(counter), 'w') as fp:
            fp.write('\n'.join(all_text))

    def sample_batch():
        return [data_sampler.sample(1024) for _ in range(batch_size)]

    if overwrite and restore_from == 'latest':
        for file in files:
            if file.startswith('model') or file.startswith('events'):
                os.remove(os.path.join(checkpoint_path, file))
        save()

    avg_loss = (0.0, 0.0)
    start_time = time.time()

    if steps:
        steps = int(steps)
    
    try:
        while True:
            if steps > 0 and counter == (counter_base + steps):
                save()
                return
            if (counter - 1) % save_every == 0 and counter > 1:
                save()
            if (counter - 1) % sample_every == 0 and counter > 1:
                generate_samples()

            if accumulate_gradients > 1:
                sess.run(opt_reset)
                for _ in range(accumulate_gradients):
                    sess.run(
                        opt_compute, feed_dict={context: sample_batch()})
                (v_loss, v_summary) = sess.run((opt_apply, summary_loss))
            else:
                (_, v_loss, v_summary) = sess.run(
                    (opt_apply, loss, summary_loss),
                    feed_dict={context: sample_batch()})

            summary_log.add_summary(v_summary, counter)

            if counter % print_every == 0:
                avg_loss = (avg_loss[0] * 0.99 + v_loss,
                            avg_loss[1] * 0.99 + 1.0)
                avg=avg_loss[0] / avg_loss[1]
                if avg  < 1.50:
                  save()
                  gpt2.copy_checkpoint_to_gdrive(run_name=run_name)
                  print('Model Trained.')
                  break
                print(
                    '[{counter} | {time:2.2f}] loss={loss:2.2f} avg={avg:2.2f}'
                    .format(
                        counter=counter,
                        time=time.time() - start_time,
                        loss=v_loss,
                        avg=avg_loss[0] / avg_loss[1]))

            counter += 1
    except KeyboardInterrupt:
        print('interrupted')
        save()


In [ ]:
def open_scripts(start, end):
  """
  Opens and seperates scripts into 4 equal parts for training. 
  Removes corrupted bytes and minorly scrubs data. 
  """
  a = ''
  b = ''
  c = ''
  d = ''
  rng = [i for i in range(start, end)]
  for x in [15, 354, 377, 378, 379, 380, 381, 382, 383, 384, 385, 
            386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 
            397, 398, 399, 400, 401, 473, 610, 718]:
    if x in rng:
      rng.remove(x)
  txts = ''
  for i in rng:
    txt = open(f"/content/drive/MyDrive/trekgenerator/{i}.txt",
                "r", encoding='utf-8', errors='ignore')
    txt = txt.read()
    txt = txt.replace("\t", " ").replace('...', ' ').replace('..', ' ')
    txt = txt.replace('                                           ',
                      ' ').replace('\\', '').replace('--', '-')
    txt = txt.replace('       ', ' ').strip().replace('      ', 
                      ' ').replace('     ', ' ')
    txt = txt.replace('    ', ' ').replace('   ',
                      ' ').replace('   ', ' ').replace('  ', ' ')
    if 'PART ONE' in txt:
      txt = txt[txt.find('PART ONE'):]
      part_one = txt[txt.find('PART ONE'):txt.find('PART TWO')]
      part_two = txt[txt.find('PART TWO'):txt.find('PART THREE')]
      part_three = txt[txt.find('PART THREE'):txt.find('PART FOUR')]
      part_four = txt[txt.find('PART FOUR'):]
    elif 'PART FIVE' in txt:
      txt = txt[txt.find('PART FIVE'):]
      part_one = txt[txt.find('PART FIVE'):txt.find('PART SIX')]
      part_two = txt[txt.find('PART SIX'):txt.find('PART SEVEN')]
      part_three = txt[txt.find('PART SEVEN'):txt.find('PART EIGHT')]
      part_four = txt[txt.find('PART EIGHT'):]
    elif 'ACT ONE' in txt:
      txt = txt[txt.find('ACT ONE'):]
      part_one = txt[txt.find('ACT ONE'):txt.find('ACT TWO')]
      part_two = txt[txt.find('ACT TWO'):txt.find('ACT THREE')]
      part_three = txt[txt.find('ACT THREE'):txt.find('ACT FOUR')]
      part_four = txt[txt.find('ACT FOUR'):]
    elif 'ACT FIVE' in txt:
      txt = txt[txt.find('ACT FIVE'):]
      part_one = txt[txt.find('ACT FIVE'):txt.find('ACT SIX')]
      part_two = txt[txt.find('ACT SIX'):txt.find('ACT SEVEN')]
      part_three = txt[txt.find('ACT SEVEN'):txt.find('ACT EIGHT')]
      part_four = txt[txt.find('ACT EIGHT'):]
    a += part_one.strip() + ' <|endoftext|> \n <|startoftext|> '
    b += part_two.strip() + ' <|endoftext|> \n <|startoftext|> '
    c += part_three.strip() + ' <|endoftext|> \n <|startoftext|> '
    d += part_four.strip() + ' <|endoftext|> \n <|startoftext|> '
    txts += txt.strip() + ' <|endoftext|> \n <|startoftext|> '
  return a, b, c, d, txts 
a, b, c, d , txts = open_scripts(0, 826)

In [ ]:
for letter in [a, b, c, d]:
  letter = letter.replace('+', ' ').replace('\x00', '').replace('\n', ' \n ').strip()
  letter = letter.replace('[', '(').replace(']', ')').replace('{', '(').replace('}', ')')
  letter = letter.replace('((', '(').replace('))', ')')[:-17]
  for key in names.keys():
    letter = letter.replace(key, names[key])
  letter = letter.split('\n')
  letter = pd.DataFrame(letter)
  letter = letter.loc[letter[0] != '']
  letter = letter.rename(columns={0: 'quote'})
  letter.reset_index(inplace=True, drop=True)
  letter = letter[letter['quote'] != ':']
  letter = letter[letter['quote'] != '-']
  letter = letter[letter['quote'] != '\n']
  letter = letter[letter['quote'] != '.']
  letter = letter.loc[letter['quote'] != '']
  letter = letter.loc[letter['quote'] != ' ']
  prev = 0 
  letter.reset_index(inplace=True, drop=True)
  for i in range(len(letter)):
    string = ''
    bool1 = any(ext in letter.at[i, 'quote'] for ext in to_drop)
    bool2 = letter.at[i, 'quote'].replace('\n', '').strip()[:-1].isnumeric()
    bool3 = len(letter.at[i, 'quote']) < 6
    if bool1 or bool2 or bool3:
      letter.drop(index=i, inplace = True)
    else:
      letter.at[i, 'quote'] = letter.at[i, 'quote'].replace('\n', '').strip()
      caps = letter.at[i, 'quote'].isupper()
      sentence = letter.at[i, 'quote'].split()
      num = 0
      for j in sentence:
        if j.isupper() and num <= 2 and j.startswith('(') == False:
          string += j + ' '
          num += 1
        else:
          string += j.lower() + ' '
          num += 1
      if '(FORMERLY' in string:
        string = string[:string.find('(FORMERLY')]
      end = letter.at[i, 'quote'].strip()[-1:]
      if end in ['.', '?', '(', ')', '!'] or caps and len(sentence) >= 2:
        end = ' \n '
      else:
        end = ''
      letter.at[i, 'quote'] = string + end
  letter.reset_index(inplace=True, drop=True)
  A = ''
  for i in a.index:
    quote = letter.at[i, "quote"]
    while quote[:1].isnumeric() or quote[:1] == '-' or quote[:1] == '.':
      quote = quote[1:]
    for i in quote:
      if i.isalnum() == False:
        if i not in [',', '.', '?', '(', ')', '-', '!']:
          quote.replace(f'{i}', '')
    quote.replace('---', '').replace('--', '')
    A += quote + ' '
  A = A.replace('--', '').replace(' \n  \n ', ' \n ').replace('  ', ' ')
  with open(f'/content/drive/MyDrive{letter}.txt', 'w') as f:
      f.write(A)